---
sidebar_label: NimbleIt Retriever
---

### Installation

You will need to install both the `langchain_community` pacakges:

In [ ]:
%pip install -qU langchain_community

# NimbleIt Retriever

In [2]:
import getpass
import os

if "NIMBLE_API_KEY" not in os.environ:
    os.environ["NIMBLE_API_KEY"] = getpass.getpass("Enter your NIMBLE_API_KEY: ")

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

### Instantiation
Now we can instantiate our retriever, initializing a client and optionally pre-adding data sources urls

In [3]:
from langchain_community.retrievers.nimbleit import (
    NimbleItRetriever,
)

retriever = NimbleItRetriever(
    answerit_url="https://answerit.webit.live",
    authorization_token=os.environ["NIMBLE_API_KEY"],
)

In [4]:
print('pipeline_id', retriever.store.pipeline_id)

pipeline_id 250a4459-5d25-4cbb-8248-598a3e924d0a


#### Add Sources of Information from URLs

In [5]:
retriever.add_sources(
    [
        {
            "url": "https://lilianweng.github.io/posts/2023-06-23-agent/",
            "return_most_relevant_content": False,
            "depth": 0
        },
        {
            "url": "https://en.wikipedia.org/wiki/Pizza",
            "return_most_relevant_content": False,
            "depth": 0
        }
    ],
    wait=True
 )

In [6]:
result = retriever.invoke("What is llm?")
print(result)

[Document(metadata={'evidences': [{'evidence': 'They did an experiment on fine-tuning LLM to call a calculator, using arithmetic as a test case.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'ChatGPT Plugins and OpenAI API function calling are good examples of LLMs augmented with tool use capability working in practice.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'HuggingGPT (Shen et al. 2023) is a framework to use ChatGPT as the task planner to select models available in HuggingFace platform according to the model descriptions and summarize the response based on the execution results.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, {'evidence': 'LLM+P (Liu et al. 2023), involves relying on an external classical planner to do long-horizon planning.', 'most_relevant_content': '', 'url': 'https://lilianweng.github.io/

### Use within a chain
Initialize ChatOpenAI and define a prompt template



In [ ]:
!pip install langchain_openai

In [152]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


llm = ChatOpenAI()

PROMPT_TEMPLATE = """
Human: You are an AI assistant that shall provide a email letter
with official formating that
concludes the context of the <question> and <answer> with it's <reference>.

<question>
{question}
</question>

<answer>
{answer}
</answer>

Assistant:"""

prompt = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["question", "answer"]
)

#### Define a function for formatting documents



In [153]:
def format_docs(docs):
    r = "\n\n".join([
        str(doc.page_content + 
        "\n".join(
            [
                f'<reference>\n {e["url"]} - {e["evidence"]}\n</reference>'
                for e in doc.metadata['evidences']
            ]
        ))
        for doc in docs
        if doc.page_content != "UNANSWERABLE"
    ])
    return r

#### Define a chain using the retriever and other components

In [154]:
retriever = NimbleItRetriever(
    answerit_url="https://answerit.webit.live",
    authorization_token=os.environ["NIMBLE_API_KEY"],
)
rag_chain = (
    {"question": RunnablePassthrough(), "answer": retriever | format_docs}
    | prompt
    | llm
    | StrOutputParser()
)

#### Add sources of information

In [155]:
retriever.add_sources([
    {
        'url': 'https://en.wikipedia.org/wiki/Lila_(Robinson_novel)',
        'depth': 0
    },
    {
        'url': 'https://www.newyorker.com/magazine/2014/10/06/lonesome-road',
        'depth': 0
    }
], wait=True)

#### Perform a query using the defined chain

In [156]:
result = rag_chain.invoke(
    "What novels has Lila written and what are their contents?"
)
result

"Dear [Recipient],\n\nI am writing to provide information regarding the novels written by Marilynne Robinson, particularly focusing on the novel 'Lila' and its contents.\n\nMarilynne Robinson has written several novels, including 'Housekeeping' (1980), 'Gilead' (2004), 'Home' (2008), 'Lila' (2014), and 'Jack' (2020). 'Lila' is the third installment of the Gilead series and explores the courtship and marriage of Lila and John Ames, as well as delving into Lila's transient past and her complex attachments. The novel addresses themes such as suffering, abandonment, forgiveness, and rescue.\n\nFor further details, you can refer to the following sources:\n1. Wikipedia: https://en.wikipedia.org/wiki/Lila_(Robinson_novel) - Lila is a novel written by Marilynne Robinson that was published in 2014. Her fourth novel, it is the third installment of the Gilead series.\n2. The New Yorker: https://www.newyorker.com/magazine/2014/10/06/lonesome-road - In 'Lila,' the author explores themes of sufferin

In [157]:
from textwrap import wrap
print('\n'.join('\n'.join(wrap(t, 70)) for t in result.split('\n')))

Dear [Recipient],

I am writing to provide information regarding the novels written by
Marilynne Robinson, particularly focusing on the novel 'Lila' and its
contents.

Marilynne Robinson has written several novels, including
'Housekeeping' (1980), 'Gilead' (2004), 'Home' (2008), 'Lila' (2014),
and 'Jack' (2020). 'Lila' is the third installment of the Gilead
series and explores the courtship and marriage of Lila and John Ames,
as well as delving into Lila's transient past and her complex
attachments. The novel addresses themes such as suffering,
abandonment, forgiveness, and rescue.

For further details, you can refer to the following sources:
1. Wikipedia: https://en.wikipedia.org/wiki/Lila_(Robinson_novel) -
Lila is a novel written by Marilynne Robinson that was published in
2014. Her fourth novel, it is the third installment of the Gilead
series.
2. The New Yorker:
https://www.newyorker.com/magazine/2014/10/06/lonesome-road - In
'Lila,' the author explores themes of suffering, abando